# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,albany,42.6001,-73.9662,3.74,80,83,1.74,US,1702775306
1,1,san antonio de pale,-1.4014,5.6325,26.98,81,93,1.65,GQ,1702775351
2,2,male,4.1748,73.5089,27.84,71,100,1.49,MV,1702775351
3,3,gavrilov posad,56.5597,40.1175,-13.94,99,70,3.10,RU,1702775351
4,4,port mathurin,-19.6833,63.4167,26.36,79,13,6.18,MU,1702775352


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
# %%capture --no-display
# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriStreet",
    frame_width = 800,
    frame_height = 600,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
list(city_data_df)

['City_ID',
 'City',
 'Lat',
 'Lng',
 'Max Temp',
 'Humidity',
 'Cloudiness',
 'Wind Speed',
 'Country',
 'Date']

In [5]:
city_data_df["City_ID"].count()

568

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df.notnull()

# Drop any rows with null values
city_df_notnull = city_data_df.dropna()
# Display sample data
city_df_notnull.head(1)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,albany,42.6001,-73.9662,3.74,80,83,1.74,US,1702775306


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
selected_columns = ["City", "Country", "Lat", "Lng", "Humidity"]
hotel_df = city_df_notnull[selected_columns].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = "" #Must always add a value into the column " " is needed to be null values\

# Display sample data
hotel_df.head(5)

,City,Country,Lat,Lng,Humidity,Hotel Name
0,albany,US,42.6001,-73.9662,80,
1,san antonio de pale,GQ,-1.4014,5.6325,81,
2,male,MV,4.1748,73.5089,71,
3,gavrilov posad,RU,56.5597,40.1175,99,
4,port mathurin,MU,-19.6833,63.4167,79,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
conditions = "access"
limit = 20  

# Parameters dictionary
params = {
    "categories": categories,
    "conditions": conditions,
    "apiKey": geoapify_key,
    "limit": limit
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.head(20).iterrows(): # I had to do the .head(20) method on the dataframe because the limit paramter was not working
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()

    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
            # If no hotel is found, set the hotel name as "No hotel found".
         hotel_df.loc[index, "Hotel Name"] = "No hotel found"
  
    # Log the search results
    print(f"{row['City']}, {row['Country']} - nearest hotel: {row['Hotel Name']}")

# Display sample data
hotel_df.head()


Starting hotel search
albany, US - nearest hotel: No hotel found
san antonio de pale, GQ - nearest hotel: No hotel found
male, MV - nearest hotel: Champa Central Hotel
gavrilov posad, RU - nearest hotel: No hotel found
port mathurin, MU - nearest hotel: Escale Vacances
port-aux-francais, TF - nearest hotel: Keravel
aasiaat, GL - nearest hotel: SØMA
porto novo, BJ - nearest hotel: Hôtel 6500 CFA
thompson, CA - nearest hotel: Thompson Inn
edinburgh of the seven seas, SH - nearest hotel: No hotel found
adamstown, PN - nearest hotel: No hotel found
waitangi, NZ - nearest hotel: Hotel Chathams
khandyga, RU - nearest hotel: Апельсин
tshabong, BW - nearest hotel: No hotel found
invercargill, NZ - nearest hotel: The Grand
horta, PT - nearest hotel: Azoris Faial Garden - Resort Hotel
grytviken, GS - nearest hotel: No hotel found
vorgashor, RU - nearest hotel: София
mpulungu, ZM - nearest hotel: No hotel found
bamboo flat, IN - nearest hotel: Hotel Megapode Nest


,City,Country,Lat,Lng,Humidity,Hotel Name
0,albany,US,42.6001,-73.9662,80,No hotel found
1,san antonio de pale,GQ,-1.4014,5.6325,81,No hotel found
2,male,MV,4.1748,73.5089,71,Champa Central Hotel
3,gavrilov posad,RU,56.5597,40.1175,99,No hotel found
4,port mathurin,MU,-19.6833,63.4167,79,Escale Vacances


In [16]:
name_address

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'name': 'Hotel Megapode Nest',
    'country': 'India',
    'country_code': 'in',
    'state': 'Andaman and Nicobar Islands',
    'state_district': 'South Andaman District',
    'county': 'Port Blair Tehsil',
    'city': 'Port Blair',
    'postcode': '744102',
    'street': 'Foreshore Road',
    'lon': 92.73004,
    'lat': 11.67707,
    'state_code': 'AN',
    'formatted': 'Hotel Megapode Nest, Foreshore Road, Port Blair - 744102, Andaman and Nicobar Islands, India',
    'address_line1': 'Hotel Megapode Nest',
    'address_line2': 'Foreshore Road, Port Blair - 744102, Andaman and Nicobar Islands, India',
    'categories': ['accommodation', 'accommodation.hotel'],
    'details': [],
    'datasource': {'sourcename': 'openstreetmap',
     'attribution': '© OpenStreetMap contributors',
     'license': 'Open Database Licence',
     'url': 'https://www.openstreetmap.org/copyright',
     'raw': {'name': 'Hotel Me

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:
# Configure the map plot
map_plot_2 = hotel_df.head(50).hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriStreet",
    frame_width = 800,
    frame_height = 600,
    color = "City",
    hover = True,
    hover_cols='all'
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,index,Country,Humidity,Hotel Name)